## 2.2. Komponenty biblioteki LangChain

### Instalacja

In [ ]:
!pip install -q python-dotenv langchain langchain-openai langchain-community langchain-text-splitters faiss-cpu

### Konfiguracja .env i modelu

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś pomocnym asystentem. Odpowiadaj zwięźle."),
    ("user", "Streszcz w 1 zdaniu: {tekst}")
])

chain = prompt | llm | StrOutputParser()  # LCEL: prompt → model → parser
wynik = chain.invoke({"tekst": "LangChain ułatwia budowę aplikacji LLM, dostarczając klocki do promptów, pamięci, narzędzi i RAG."})
print(wynik)


### Prompt template

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o")
template = """
Specify tags and the main topic of the text.
tags: What are best tags describing text. Give maximum 5 tags separated by comma.
topic: What is the topic of text. Use maximum couple of words

Format response as JSON as below:
'tags': ['sometag', 'othertag', 'anothertag', 'tag4', 'tag5']
'subject': 'Some subject of text'

text: {input}
"""

prompt_template = ChatPromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.predict(input="They picked a way among the trees, and their ponies plodded along, carefully avoiding the many writhing and interlacing roots.  There was no undergrowth.  The ground was rising steadily, and as they went forward it seemed that the trees became taller, darker, and thicker. There was no sound, except an occasional drip of moisture falling through the still leaves.  For the moment there was no whispering or movement among the branches; but they all got an uncomfortable feeling that they were being watched with disapproval, deepening to dislike and even enmity.  The feeling steadily grew, until they found themselves looking up quickly, or glancing back over their shoulders, as if they expected a sudden blow.")

### ResponseSchema i OutputParser

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

tags_schema = ResponseSchema(
    name="tags",
    description=" What are best tags describing text. Give maximum 5 tags separated by comma.",
)
topic_schema = ResponseSchema(
    name="topic",
    description="What is the topic of text. Use maximum couple of words."
)

response_schemas = [tags_schema, topic_schema]
parser = StructuredOutputParser.from_response_schemas(response_schemas)
template = """
Specify tags and the main topic of the text.
tags: What are best tags describing text. Give maximum 5 tags separated by comma.
topic: What is the topic of text. Use maximum couple of words

Format response as JSON as below:
'tags': ['sometag', 'othertag', 'anothertag', 'tag4', 'tag5']
'subject': 'Some subject of text'

text: {input}

{instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template)
instructions = parser.get_format_instructions()

messages = prompt.format_messages(
    input="They picked a way among the trees, and their ponies plodded along, carefully avoiding the many writhing and interlacing roots.  There was no undergrowth.  The ground was rising steadily, and as they went forward it seemed that the trees became taller, darker, and thicker. There was no sound, except an occasional drip of moisture falling through the still leaves.  For the moment there was no whispering or movement among the branches; but they all got an uncomfortable feeling that they were being watched with disapproval, deepening to dislike and even enmity.  The feeling steadily grew, until they found themselves looking up quickly, or glancing back over their shoulders, as if they expected a sudden blow.",
    instructions=instructions,
)

chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response)
output_dict = parser.parse(response.content)
print(output_dict)

### Prompty, Parsowanie, LCEL / Chains
PromptTemplate + LLM + StrOutputParser (LCEL)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś pomocnym asystentem. Odpowiadaj zwięźle."),
    ("user", "Streszcz w 1 zdaniu: {tekst}")
])

chain = prompt | llm | StrOutputParser()  # LCEL: prompt → model → parser
wynik = chain.invoke({"tekst": "LangChain ułatwia budowę aplikacji LLM, dostarczając klocki do promptów, pamięci, narzędzi i RAG."})
print(wynik)


### Tools (narzędzia, które może wywołać model)
prosty tool (kalkulator) + agent ReAct

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: float, b: float) -> float:
    """Zwraca sumę a+b."""
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Zwraca iloczyn a*b."""
    return a * b

tools = [add, multiply]

from langchain import agents
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

react_prompt = PromptTemplate.from_template(
"""Jesteś pomocnym asystentem. Masz dostęp do narzędzi.
Używaj ich tylko gdy potrzebne. Odpowiadaj po polsku.

Pytanie: {input}
{agent_scratchpad}"""
)

agent = create_react_agent(llm, tools, react_prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = executor.invoke({"input": "Policz (12 + 7) * 3 i podaj wynik."})
print("\nWynik końcowy:", response["output"])


### Memory (historia rozmowy / stan)
RunnableWithMessageHistory (pamięć czatu)

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Prowadzisz przyjazną rozmowę i pamiętasz kontekst."),
    ("user", "{input}")
])

chat_chain = chat_prompt | llm | StrOutputParser()

store = {}  # prosta “baza” historii po session_id

def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_with_memory = RunnableWithMessageHistory(
    chat_chain,
    get_history,
    input_messages_key="input",
    history_messages_key="history",
)

sid = "demo-session-1"
print(chain_with_memory.invoke({"input": "Cześć! Mam na imię Michał."},
                               config={"configurable": {"session_id": sid}}))
print(chain_with_memory.invoke({"input": "Jak mam na imię?"},
                               config={"configurable": {"session_id": sid}}))


### History

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
# message history
history = ChatMessageHistory()

history.add_user_message("Buenos dias!")
history.add_ai_message("hello!")
history.add_user_message("Whats your name?")
history.add_ai_message("My name is GIGACHAT")
history.messages

### Memory

In [ ]:
# memory
# History keeps all messages between the user and AI intact. History is what the user sees in the UI.
# It represents what was actually said. Memory keeps some information, which is presented to the LLM to make it behave as if it "remembers" the conversation.
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("Buenos dias!")
memory.chat_memory.add_ai_message("Hello!")
memory.chat_memory.add_user_message("Whats your name?")
memory.chat_memory.add_ai_message("My name is GIGACHAT")
memory.load_memory_variables({})

In [ ]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, verbose=True, memory=memory
)
conversation.predict(input="Buenos Dias!") # will it response in different language?

### Summary

In [ ]:
!pip install tiktoken

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

previous_output_review = "Rdr2 is an experience, as it's more than just a video game. It's like being in one long Oscar winning movie. Script, acting, storyline are immense, it's quite unbelievable really. You are living as a cowboy day in day out eating, bathing, sleeping, everything in a huge openworld."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=1-0)
memory.save_context(
    {"input": f"Could you analyze review for me {previous_output_review}?"},
    {"output": "Sure, no problem"},
)

In [ ]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [ ]:
conversation.predict(input="Thank you")